In [14]:
import pandas as pd
import numpy as np
import tensorflow as tf
from transformers import AutoModel , AutoImageProcessor , AutoTokenizer 
from datasets import load_dataset , Dataset
from datasets import Image as dImage
from PIL import Image
import torch
from torch import nn

torch.cuda.empty_cache()

# import os
# os.environ['TF_GPU_ALLOCATOR']='cuda_malloc_async'

2024-05-16 12:42:02.774786: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-16 12:42:03.349117: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/user/miniconda3/envs/python38/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [28]:
df_acc = pd.read_pickle("./vqa_v2_acc.pkl")
df_acc.head()

,question,question_type,question_id,image_id,answer_type,label
0,What is this photo taken looking through?,what is this,458752000,COCO_train2014_000000458752.jpg,other,"{'ids': ['net', 'netting', 'mesh'], 'weights':..."
1,What position is this man playing?,what,458752001,COCO_train2014_000000458752.jpg,other,"{'ids': ['pitcher', 'catcher'], 'weights': [0...."
2,What color is the players shirt?,what color is the,458752002,COCO_train2014_000000458752.jpg,other,"{'ids': ['orange'], 'weights': [1.0]}"
3,Is this man a professional baseball player?,is this,458752003,COCO_train2014_000000458752.jpg,yes/no,"{'ids': ['yes', 'no'], 'weights': [0.94, 0.06]}"
4,What color is the snow?,what color is the,262146000,COCO_train2014_000000262146.jpg,other,"{'ids': ['white'], 'weights': [1.0]}"


In [29]:
np.array(Image.open("./train2014_3d/" + df_acc["image_id"][0])).shape

(480, 640, 3)

In [105]:
# SAMPLES = 100
# df = df_acc[:SAMPLES]

df = df_acc.groupby('question_type').sample(n=200 , replace=True)
df = df.reset_index(drop=True)
SAMPLES = len(df)

print("Number of samples are : " , SAMPLES)

df.head()

Number of samples are :  13000


,question,question_type,question_id,image_id,answer_type,label
0,Are all the men using headphones?,are,448439000,COCO_train2014_000000448439.jpg,yes/no,"{'ids': ['no'], 'weights': [1.0]}"
1,Are any of these ties tied?,are,21674004,COCO_train2014_000000021674.jpg,yes/no,"{'ids': ['yes', 'no'], 'weights': [0.79, 0.21]}"
2,Are all of these adults?,are,532439000,COCO_train2014_000000532439.jpg,yes/no,"{'ids': ['yes', 'no'], 'weights': [0.8, 0.2]}"
3,Are both doors open on the icebox?,are,341218001,COCO_train2014_000000341218.jpg,yes/no,"{'ids': ['yes'], 'weights': [1.0]}"
4,Are those mountains in the distance?,are,23249002,COCO_train2014_000000023249.jpg,yes/no,"{'ids': ['yes', 'no'], 'weights': [0.95, 0.05]}"


Creating Labels

In [106]:
import itertools

labels = [item['ids'] for item in df['label'].to_list()]
flattened_labels = list(itertools.chain(*labels))
unique_labels = list(set(flattened_labels))

label2id = {label: idx for idx, label in enumerate(unique_labels)}
id2label = {idx: label for label, idx in label2id.items()} 

In [107]:
label2id

{'hard to read': 0,
 'over shoulder': 1,
 '$4.20': 2,
 'near people': 3,
 'raspberry tart': 4,
 'cutlery': 5,
 'pudding': 6,
 'guitar': 7,
 'pizza pan': 8,
 'fake tree': 9,
 '2:43': 10,
 'oldest man': 11,
 'humming bird and bird are stop action photo': 12,
 'asian': 13,
 'lab': 14,
 'pound symbol': 15,
 'under wave': 16,
 'buildings': 17,
 "in girl's mouth": 18,
 'acoustic': 19,
 'in': 20,
 'cleanse hands': 21,
 'yacht': 22,
 'blue bird': 23,
 'pavement': 24,
 'it is tennis racquet in her hand': 25,
 'seed packet': 26,
 'on top of train': 27,
 'women, bags': 28,
 'because of heat': 29,
 'slow': 30,
 '1 and 2': 31,
 'pants and dress': 32,
 'valparaiso': 33,
 '11:55': 34,
 'fall': 35,
 'carmel': 36,
 '12:30': 37,
 '1 hour': 38,
 'its black and white': 39,
 'bank': 40,
 'outhouse': 41,
 'succotash': 42,
 'construction site': 43,
 'hurst grove, street pride': 44,
 'elderly gentleman': 45,
 'accents': 46,
 'for storage': 47,
 'hp': 48,
 'pull': 49,
 'bicycle proceed': 50,
 'stairway': 51,
 

In [109]:
NUM_CLASSES = len(id2label)
print("Number of Classes : " , NUM_CLASSES)

Number of Classes :  12251


In [110]:
n = SAMPLES
rf = 10 ** (len(str(n))-1)
r = str(int(np.round(n / rf) * rf))

r = r[:-3] + r[-3:].replace("000" , "k")

pd.to_pickle(df , f"./vqa_v2_sampled_{r}.pkl")

### Evaluate Existing Dataset

In [111]:
ds = pd.read_pickle("vqa_v2_sampled_10k.pkl")
# ds = df_acc.groupby('question_type').sample(n=200 , replace=True)
# ds = ds.reset_index(drop=True)

SAMPLES = len(ds)
print("Number of samples are : " , SAMPLES)

labels = [item['ids'] for item in ds['label'].to_list()]
flattened_labels = list(itertools.chain(*labels))
unique_labels = list(set(flattened_labels))

label2id = {label: idx for idx, label in enumerate(unique_labels)}
id2label = {idx: label for label, idx in label2id.items()} 

NUM_CLASSES = len(id2label)
print("Number of Classes : " , NUM_CLASSES)

Number of samples are :  13000
Number of Classes :  12251
